In [46]:
from pathlib import Path
import shutil
from tqdm import tqdm
import pandas as pd

num_obj_train = 4
num_obj_val = 4
num_obj_test = 2
k = num_obj_train  + num_obj_test + num_obj_val

root_target = Path(f"/home/borth/sketch2shape/data/shapenet_chair_{k}")
root_siamese = Path("/home/borth/sketch2shape/data/shapenet_chair_large/")
root_source = Path("/shared/data")

shapenet_id = "03001627"
split_folder = "overfit_16"

obj_ids = []
for obj_file_paths in (root_source / "ShapeNetV2/SdfSamples" / split_folder / shapenet_id).iterdir():
    obj_id = obj_file_paths.stem
    if (root_siamese / "shapes" / obj_id).exists():
        obj_ids.append(obj_id)

for obj_id in tqdm(obj_ids):
    source2target = {
        root_source / "ShapeNetV2/NormalizationParameters" / split_folder / shapenet_id / f"{obj_id}.npz": "normalization.npz",
        root_source / "ShapeNetV2/SurfaceSamples" / split_folder / shapenet_id / f"{obj_id}.ply": "surfaces_samples.ply",
        root_source / "ShapeNetV2/SdfSamples" / split_folder / shapenet_id / f"{obj_id}.npz": "sdf_samples.npz",
        root_source / "ShapeNetCore" / shapenet_id / obj_id / "models/model_normalized.obj": "model_normalized.obj",
    }
    for source_path, target_file in source2target.items():
        target = root_target / "shapes" / obj_id / target_file 
        target.parent.mkdir(parents=True, exist_ok=True)
        shutil.copy(source_path, target)
    
for obj_id in tqdm(obj_ids):
    source2target = {
        root_siamese / "shapes" / obj_id / "images": "images",
        root_siamese / "shapes" / obj_id / "sketches": "sketches",
    }
    for source_folder, target_dir in source2target.items():
        target = root_target / "shapes" / obj_id / target_dir 
        if target.exists():
            shutil.rmtree(target)
        shutil.copytree(source_folder, target)

data = []
splits = ["train"] * 4 + ["val"] * 4 + ["test"] * 2
for label, obj_id in enumerate(obj_ids):
    data.append({"obj_id": obj_id, "label": label, "split": splits[label]})
df = pd.DataFrame(data)
df.to_csv(root_target / "metainfo.csv", index=False)
# folder = "NormalizationParameters"
# for files in Path(path / folder).iterdir():
#     print(files)

  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 10/10 [00:00<00:00, 96.79it/s]


In [43]:
df

,obj_id,label,split
0,7e81b5f79e6899cea570c6c691c987a8,0,train
1,787a4db5b3452fc357a847db7547c1f3,1,train
2,88042e6d331ffba0f63e8e405d49b8ce,2,train
3,6be6d6ae38d8aca4dc2cbc0befb06e1b,3,train
4,8c7d2eab593269f8607566378b3d8827,4,val
5,78ba9e32a9a5c276f69d13812ce67f33,5,val
6,2314a3400230f50d14038d588fd1342f,6,val
7,780809a0d1b68f4a8ef4ac3a24abb05b,7,val
8,3fdc09d3065fa3c524e7e8a625efb2a7,8,test
9,2c67f62020ca22927297db8481c2b16b,9,test


In [36]:
source_target = Path("/home/borth/sketch2shape/data/shapenet_chair_large/shapes")
for obj_id in tqdm(obj_ids):
    obj_folder = source_target / obj_id
    print(obj_folder.exists())

100%|██████████| 16/16 [00:00<00:00, 24947.53it/s]

True
True
True
True
False
False
True
False
False
True
False
True
True
True
False
True


In [4]:
from lib.data.metainfo import MetaInfo


metainfo = MetaInfo(data_dir="/home/borth/sketch2shape/data/shapenet_chair_10", split="val")
metainfo._sketch_image_pairs

,obj_id,image_id,label
0,8c7d2eab593269f8607566378b3d8827,00000,4
1,8c7d2eab593269f8607566378b3d8827,00001,4
2,8c7d2eab593269f8607566378b3d8827,00002,4
3,8c7d2eab593269f8607566378b3d8827,00003,4
4,8c7d2eab593269f8607566378b3d8827,00004,4
...,...,...,...
123,780809a0d1b68f4a8ef4ac3a24abb05b,00027,7
124,780809a0d1b68f4a8ef4ac3a24abb05b,00028,7
125,780809a0d1b68f4a8ef4ac3a24abb05b,00029,7
126,780809a0d1b68f4a8ef4ac3a24abb05b,00030,7


{'obj_id': '8c7d2eab593269f8607566378b3d8827', 'image_id': '00000', 'label': 4}